# Featureモジュールの解説と実行

In [2]:
#パッケージのインポート
import glob
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models,transforms
import cv2 
import time

## FeatureMap_convolutionの実装

In [2]:
class conv2DBatchNormRelu(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size,stride,padding,dilation,bias):
        super(conv2DBatchNormRelu,self).__init__()
        self.conv=nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding,dilation,bias=bias)
        self.batchnorm=nn.BatchNorm2d(out_channels)
        self.relu=nn.ReLU(inplace=True)
        #inplace設定で、入力を保存せずに出力を計算し、メモリ削減する

    def forward(self,x):
        x=self.conv(x)
        x=self.batchnorm(x)
        outputs=self.relu(x)

        return outputs
    

In [4]:
class FeatureMap_convolution(nn.Module):
    def __init__(self):
        """ 構成するネットワークを用意 """
        super(FeatureMap_convolution,self).__init__()

        #畳み込み層1
        in_channels,out_channels,kernel_size,stride,padding,dilation,bias = 3,64,3,2,1,1,False
        self.cbnr_1=conv2DBatchNormRelu(in_channels,out_channels,kernel_size,stride,padding,dilation,bias)

        #畳み込み層2
        in_channels,out_channels,kernel_size,stride,padding,dilation,bias=64,64,3,1,1,1,False
        self.cbnr_2=conv2DBatchNormRelu(in_channels,out_channels,kernel_size,stride,padding,dilation,bias)

        #畳み込み層3
        in_channels,out_channels,kernel_size,stride,padding,dilation,bias=64,128,3,1,1,1,False
        self.cbnr_3=conv2DBatchNormRelu(in_channels,out_channels,kernel_size,stride,padding,dilation,bias)

        #MaxPooling層
        self.maxpool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)

    def forward(self,x):
        x=self.cbnr_1(x)
        x=self.cbnr_2(x)
        x=self.cbnr_3(x)
        outputs=self.maxpool(x)
        return outputs
    

## ResidualBlockPSP

In [3]:
class ResidualBlockPSP(nn.Sequential):
    def __init__(self,n_blocks,in_channels,mid_channels,out_channels,stride,dilation):
        super(ResidualBlockPSP,self).__init__()

        #bottleNeckPSPの用意
        self.add_module(
            "block1",
            bottleNeckPSP(in_channels,mid_channels,out_channels,stride,dilation)
        )

        #bottleNeckIdentifyPSPの繰り返しの用意
        for i in range(n_blocks-1):
            self.add_module(
                "block"+str(i+2),
                bottleNeckIdentifyPSP(out_channels,mid_channels,stride,dilation)
            )

In [5]:
class conv2DBatchNorm(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size,stride,padding,dilation,bias):
        super(conv2DBatchNorm,self).__init__()
        self.conv=nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding,dilation,bias=bias)
        self.batchnorm=nn.BatchNorm2d(out_channels)
        
    def forward(self,x):
        x=self.conv(x)
        outputs=self.batchnorm(x)

        return outputs

In [6]:
class bottleNeckPSP(nn.Module):
    def __init__(self,in_channels,mid_channels,out_channels,stride,dilation):
        super(bottleNeckPSP,self).__init__()

        self.cbr_1=conv2DBatchNormRelu(in_channels,mid_channels,kernel_size=1,stride=1,padding=0,
                                 dilation=1,bias=False)
        self.cbr_2=conv2DBatchNormRelu(mid_channels,mid_channels,kernel_size=3,stride=stride,padding=dilation,
                                 dilation=dilation,bias=False)
        self.cbr_3=conv2DBatchNorm(mid_channels,out_channels,kernel_size=1,stride=1,padding=0,
                                 dilation=1,bias=False)

        #スキップ結合
        self.cb_residual=conv2DBatchNorm(in_channels,out_channels,kernel_size=1,stride=stride,padding=0,
                                 dilation=1,bias=False)

        self.relu=nn.ReLU(inplace=True)

    def forward(self,x):
        conv=self.cbr_3(self.cbr_2(self.cbr_1))
        residual=self.cb_residual(x)
        return self.relu(conv+residual)

In [7]:
class bottleNeckIdentifyPSP(nn.Module):
    def __init__(self,in_channels,mid_channels,out_channels,stride,dilation):
        super(bottleNeckIdentifyPSP,self).__init__()

        self.cbr_1=conv2DBatchNormRelu(in_channels,mid_channels,kernel_size=1,stride=1,padding=0,
                                 dilation=1,bias=False)
        self.cbr_2=conv2DBatchNormRelu(mid_channels,mid_channels,kernel_size=3,stride=1,padding=dilation,
                                 dilation=dilation,bias=False)
        self.cbr_3=conv2DBatchNorm(mid_channels,out_channels,kernel_size=1,stride=1,padding=0,
                                 dilation=1,bias=False)
        self.relu=nn.ReLU(inplace=True)

        
    def forward(self,x):
        conv=self.cbr_3(self.cbr_2(self.cbr_1))
        residual=x
        return self.relu(conv+residual)